In [1]:
import pandas as pd
import numpy as np
import faiss

In [2]:
train_df = pd.read_csv("E:/Software/Yandex Practicum/Datasets/Masterskaya2/train.csv")
validation_df = pd.read_csv("E:/Software/Yandex Practicum/Datasets/Masterskaya2/validation.csv")

train_head = train_df.head()
validation_head = validation_df.head()

print(train_head)
print()
print(validation_head)

        Id          0          1           2          3           4  \
0  0-query -53.882748  17.971436  -42.117104 -183.93668  187.517490   
1  1-query -87.776370   6.806268  -32.054546 -177.26039  120.803330   
2  2-query -49.979565   3.841486 -116.118590 -180.40198  190.128430   
3  3-query -47.810562   9.086598 -115.401695 -121.01136   94.652840   
4  4-query -79.632126  14.442886  -58.903397 -147.05254   57.127068   

            5           6          7           8  ...         63         64  \
0  -87.144930 -347.360606  38.307602  109.085560  ...  70.107360 -155.80257   
1  -83.810590  -94.572749 -78.433090  124.915900  ...   4.669178 -151.69771   
2  -50.837620   26.943937 -30.447489  125.771164  ...  78.039764 -169.14620   
3 -109.255410 -775.150134  79.186520  124.003100  ...  44.515266 -145.41675   
4  -16.239529 -321.317964  45.984676  125.941284  ...  45.028910 -196.09207   

           65         66          67          68          69           70  \
0 -101.965943  65.903

In [3]:
validation_answer_df = pd.read_csv("E:/Software/Yandex Practicum/Datasets/Masterskaya2/validation_answer.csv")
validation_answer_head = validation_answer_df.head()

print(validation_answer_head)

             Id      Expected
0  100000-query  2676668-base
1  100001-query    91606-base
2  100002-query   472256-base
3  100003-query  3168654-base
4  100004-query    75484-base


In [4]:
base_df = pd.read_csv("E:/Software/Yandex Practicum/Datasets/Masterskaya2/base.csv")

base_head = base_df.head()
base_head

,Id,0,1,2,3,4,5,6,7,8,...,62,63,64,65,66,67,68,69,70,71
0,0-base,-115.083890,11.152912,-64.426760,-118.88089,216.482440,-104.698060,-469.070588,44.348083,120.915344,...,-42.808693,38.800827,-151.76218,-74.389090,63.66634,-4.703861,92.933610,115.269190,-112.756640,-60.830353
1,1-base,-34.562202,13.332763,-69.787610,-166.53348,57.680607,-86.098370,-85.076666,-35.637436,119.718636,...,-117.767525,41.100000,-157.82940,-94.446806,68.20211,24.346846,179.937930,116.834000,-84.888941,-59.524610
2,2-base,-54.233746,6.379371,-29.210136,-133.41383,150.895830,-99.435326,52.554795,62.381706,128.951450,...,-76.397800,46.011803,-207.14442,127.325570,65.56618,66.325680,81.073490,116.594154,-1074.464888,-32.527206
3,3-base,-87.520130,4.037884,-87.803030,-185.06763,76.369540,-58.985165,-383.182845,-33.611237,122.031910,...,-70.647940,-6.358921,-147.20105,-37.692750,66.20289,-20.566910,137.206940,117.474100,-1074.464888,-72.915490
4,4-base,-72.743850,6.522049,43.671265,-140.60803,5.820023,-112.074080,-397.711282,45.182500,122.167180,...,-57.199104,56.642403,-159.35184,85.944724,66.76632,-2.505783,65.315285,135.051590,-1074.464888,0.319401


*Наборы данных имеют следующую структуру:*  

**train.csv:**

- **Id**: уникальный идентификатор для каждого продукта в наборе данных обучения.
- **От 0 до 71**: 72-мерный вектор признаков, представляющий каждый продукт.
- **Target**: идентификатор продукта из base.csv, наиболее похожего на текущий продукт.

**validation.csv:**

- **Id**: уникальный идентификатор для каждого продукта в наборе данных проверки.
- **От 0 до 71**: 72-мерный вектор признаков, представляющий каждый продукт.

**base.csv:**

- **Id**: Уникальный идентификатор для каждого продукта в образце базового набора данных.
- **0 to 71**: 72-мерный вектор признаков, представляющий каждый продукт.

**validation_answer.csv:**

- **Id**: Уникальный идентификатор для каждого продукта в наборе данных проверки.
- **Expected**: Идентификатор продукта из base.csv, наиболее похожего на текущий продукт (true label).

Учитывая поставленную задачу, нам нужно использовать данные обучения для построения модели или индекса, который позволит нам эффективно находить ближайших соседей из base.csv для каждого продукта в validation.csv.

Мы будем использовать алгоритм приближенного поиска ближайших соседей (*ANN - Approximate Nearest Neighbor*).

# Подготовка данных

In [5]:
# Extracting feature vectors and converting them into numpy arrays
train_vectors = train_df.iloc[:, 1:-1].values.astype('float32')
train_labels = train_df['Target'].values

base_vectors = base_df.iloc[:, 1:].values.astype('float32')
base_labels = base_df['Id'].values

validation_vectors = validation_df.iloc[:, 1:].values.astype('float32')
validation_labels = validation_answer_df['Expected'].values

# Checking the shapes to ensure everything is correct
train_vectors.shape, base_vectors.shape, validation_vectors.shape

((100000, 72), (2918139, 72), (100000, 72))

In [6]:
# Number of nearest neighbors we want to retrieve
k = 5

# Step 1: Index Construction
# We'll use the IndexFlatL2 index which is a basic L2 distance index
index = faiss.IndexFlatL2(72)

# Adding our base sample vectors to the index
index.add(base_vectors)

# Step 2: Searching
# This will retrieve the top k nearest neighbors from base for each vector in validation
D, I = index.search(validation_vectors, k)

# D contains the distances and I contains the indices of the nearest neighbors
# For evaluation, we'll check how often the true nearest neighbor is in the top k results

# Convert I (indices array) to actual product Ids
predicted_labels = np.array(base_labels)[I]

# Now, we can compute the accuracy@5
accuracy_at_5 = (validation_labels[:, None] == predicted_labels).any(axis=1).mean()

accuracy_at_5

0.13286